# Simple Benchmark Analysis
This notebook demonstrates how you can analyze and plot benchmark results from a single benchmark run.
Several projects exist in the `examples` folder, but this notebook assumes we are working on the
JVM part of the `kotlin-multiplatform` project. But the same approach can be used for the other projects.

First, you need to run the benchmark. This can be done by running this command from the root of the project:

```shell
./gradlew :examples:kotlin-multiplatform:jvmBenchmark
```

Once it is completed, run this notebook, and it will automatically find the latest result.

In [16]:
%use serialization, dataframe, kandy

In [17]:
import java.nio.file.Files
import java.nio.file.attribute.BasicFileAttributes
import kotlin.io.path.exists
import kotlin.io.path.forEachDirectoryEntry
import kotlin.io.path.isDirectory
import kotlin.io.path.listDirectoryEntries
import kotlin.io.path.readText

// Find latest result file, based on the their timestamp.
val runsDir = notebook.workingDir.resolve("kotlin-multiplatform/build/reports/benchmarks/main")
val lastRunDir = runsDir.listDirectoryEntries()
    .filter { it.isDirectory() }
    .sortedByDescending { dir -> Files.readAttributes(dir, BasicFileAttributes::class.java).creationTime() }
    .first()
val outputFile = lastRunDir.resolve("jvm.json")
val benchmarkData = outputFile.readText().deserializeJson()

In [18]:
import kotlinx.serialization.json.encodeToJsonElement

// Helper class for tracking the information we need to use.
data class Benchmark(val name: String, val params: String, val score: Double, val error: Double, val unit: String)

// Split benchmark results into groups. Generally, each group consist of all tests from one test file,
// except when it is an parameterized test. In this case, each test (with all its variants) are put
// in its own group.
val benchmarkGroups = benchmarkData
    .groupBy {
        if (it.params != null) {
            it.benchmark
        } else {
            it.benchmark.substringBeforeLast(".")
        }
    }
    .mapValues { group ->
        val benchmarks = group.value.map { benchmark ->
            // Parameters are specific to each test. `deserializeJson()` will generate the appropriate data classes,
            // but for generic handling of parameters we would need to fallback to reading the JSON. In this case
            // we just handle them through the typed API.
            val paramInfo = if (benchmark.params != null) {
                val paramValues = Json.decodeFromJsonElement<Map<String, String?>>(Json.encodeToJsonElement(benchmark.params))
                paramValues.entries
                    .filter { it.value != null }
                    .sortedBy { it.key }
                    .map { "${it.key}=${it.value}" }
                    .joinToString(",")
            } else {
                ""
            }
            val name = benchmark.benchmark
            Benchmark(
                name,
                paramInfo,
                benchmark.primaryMetric.score,
                benchmark.primaryMetric.scoreError,
                benchmark.primaryMetric.scoreUnit
            )
        }
        benchmarks.toDataFrame()
    }

// Un-commont this to see the benchmark data as DataFrames
// benchmarkGroups.forEach {
//     DISPLAY(it.value)
// }

In [19]:
// Prepare the data frames for plotting by:
// - Add calculated columns for errorMin / errorMax
// - Tests with parameters use the parameter values as the label
// - Tests without paramaters use the test name as the label
val plotData = benchmarkGroups.mapValues {
    it.value
        .add("errorMin") { it.getValue<Double>("score") - it.getValue<Double>("error") }
        .add("errorMax") { it.getValue<Double>("score") + it.getValue<Double>("error") }
        .insert("label") {
            // Re-format the benchmark labels to make them look "nicer"
            if (!it.getValue<String>("params").isBlank()) {
                it.getValue<String>("params").replace(",", "\n")
            } else {
                it.getValue<String>("name").substringAfterLast(".").removeSuffix("Benchmark")
            }
        }.at(0)
        .remove("name", "params")
}

In [20]:
import org.jetbrains.letsPlot.Geom
import org.jetbrains.letsPlot.core.spec.plotson.coord
import org.jetbrains.letsPlot.themes.margin

// Plot each group as a bar plot with the error displayed as error bars.
// This approach assumes that each group has tests roughly within the same "scale".
// If this is not the case, some plots might look very squished. If this happens,
// you can play around with using a LOG10 scale or modifying the limits to focus
// on the changes.
plotData.forEach { (fileName, dataframe) ->
    val plot = dataframe.plot {
        bars {
            x("label") {
                axis.name = ""
            }
            y("score")
        }
        errorBars {
            x("label")
            y("score")
            yMin("errorMin")
            yMax("errorMax")
        }
        coordinatesTransformation = CoordinatesTransformation.cartesianFlipped()
        // y.axis.limits = dataframe.min("errorMin")..dataframe.max("errorMax")
        layout {
            this.yAxisLabel = dataframe.first().getValue<String>("unit")
            style {
                global {
                    title {
                        margin(10.0, 0.0)
                    }
                    text {
                        fontFamily = FontFamily.MONO
                    }
                }
            }
            // Adjust the height of the Kandy plot based on the number of tests.
            size = 800 to ((50 * dataframe.size().nrow) + 100)
        }
    }
    DISPLAY(HTML("<h4 >$fileName</h4>"))
    DISPLAY(plot)
}

test.InheritedBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="xcgdS0"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"guides":{
"y":{
"title":"ops/s"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[999715.5730582711,1.3251985400275281E8],
"errorMax":[1024476.8651513313,1.3567233458470887E8],
"label":["base","inherited"],
"errorMin":[974954.2809652109,1.2936737342079675E8]
},
"ggsize":{
"width":800.0,
"height":200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"44"
};
 var containerDiv = document.getElementById("xcgdS0");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20M 
 
 
 
 
 
 
 40M 
 
 
 
 
 
 
 60M 
 
 
 
 
 
 
 80M 
 
 
 
 
 
 
 100M 
 
 
 
 
 
 
 120M 
 
 
 
 
 
 
 140M 
 
 
 
 
 
 
 
 
 
 
 base 
 
 
 
 
 
 
 
 
 inherited 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ops/s

test.ParamBenchmark.mathBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FMyTHu"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"guides":{
"y":{
"title":"ops/ms"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[196902.03613375648,198982.020454078,76487.5086281744,77218.5368951281],
"errorMax":[202149.32994077116,203466.72113419612,78379.96894260854,78837.91006301393],
"label":["data=1\ntext=a \"string\" with quotes\nvalue=1","data=1\ntext=a \"string\" with quotes\nvalue=2","data=2\ntext=a \"string\" with quotes\nvalue=1","data=2\ntext=a \"string\" with quotes\nvalue=2"],
"errorMin":[191654.7423267418,194497.3197739599,74595.04831374026,75599.16372724227]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"47"
};
 var containerDiv = document.getElementById("FMyTHu");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 300.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 150,000 
 
 
 
 
 
 
 200,000 
 
 
 
 
 
 
 
 
 
 
 data=1 text=a "string" with quotes value=1 
 
 
 
 
 
 
 
 
 data=1 text=a "string" with quotes value=2 
 
 
 
 
 
 
 
 
 data=2 text=a "string" with quotes value=1 
 
 
 
 
 
 
 
 
 data=2 text=a "string" with quotes value=2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ops/ms

test.ParamBenchmark.otherBenchmark

...

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="tX809v"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"guides":{
"y":{
"title":"ops/ms"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[133932.19330461326],
"errorMax":[137986.16262644873],
"label":["math"],
"errorMin":[129878.2239827778]
},
"ggsize":{
"width":800.0,
"height":150.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"56"
};
 var containerDiv = document.getElementById("tX809v");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 150.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 
 
 
 
 math 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ops/ms

test.CommonBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="GnOS3I"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"guides":{
"y":{
"title":"ms/op"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[0.0010203532398568873,2.6287927694751805E-5,7.614962302484078E-6],
"errorMax":[0.0010504284290899577,2.6979490207589775E-5,7.77349973367662E-6],
"label":["long","longBlackhole","math"],
"errorMin":[9.902780506238168E-4,2.5596365181913836E-5,7.456424871291536E-6]
},
"ggsize":{
"width":800.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"59"
};
 var containerDiv = document.getElementById("GnOS3I");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 250.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000 
 
 
 
 
 
 
 0.0001 
 
 
 
 
 
 
 0.0002 
 
 
 
 
 
 
 0.0003 
 
 
 
 
 
 
 0.0004 
 
 
 
 
 
 
 0.0005 
 
 
 
 
 
 
 0.0006 
 
 
 
 
 
 
 0.0007 
 
 
 
 
 
 
 0.0008 
 
 
 
 
 
 
 0.0009 
 
 
 
 
 
 
 0.0010 
 
 
 
 
 
 
 0.0011 
 
 
 
 
 
 
 
 
 
 
 long 
 
 
 
 
 
 
 
 
 longBlackhole 
 
 
 
 
 
 
 
 
 math 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ms/op

test.JvmTestBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="JX2WrH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"guides":{
"y":{
"title":"ns/op"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[3.9713384641892873,0.6065983726836912],
"errorMax":[4.1068243414082675,0.62305902250719],
"label":["cos","sqrt"],
"errorMin":[3.8358525869703075,0.5901377228601924]
},
"ggsize":{
"width":800.0,
"height":200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"62"
};
 var containerDiv = document.getElementById("JX2WrH");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 800.0,
 height: 200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 3.0 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 4.0 
 
 
 
 
 
 
 
 
 
 
 cos 
 
 
 
 
 
 
 
 
 sqrt 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ns/op